# Create boundary conditions for Mn extended


In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import pickle
import scipy.ndimage as ndimage
import xarray as xr

%matplotlib inline

Load extended domain

In [2]:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'])[0,:,:]
mbathy     = np.array(mesh.variables['tmask'])[0,:,:]
mesh_depth = np.array(mesh.variables['nav_lev'])

In [6]:
m      = nc.Dataset('/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2002_20211012/'+\
                    'ANHA12_EXH006_2002_monthly.nc', 'r')
mdepth = np.array(m.variables['deptht'])
mlons  = np.array(m.variables['nav_lon'])
mlats  = np.array(m.variables['nav_lat'])

Boundary coordinates:

In [7]:
imin, imax = 1099, 2179
jmin, jmax = 159, 799

isize = imax-imin
jsize = jmax-jmin

Load boundary conditions:

In [9]:
North = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/Mn_North_OBC_20210706.nc')
East  = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/Mn_East_OBC_20210706.nc')
dMn_N = North['dMn'].values
oMn_N = North['oMn'].values
dMn_E = East['dMn'].values
oMn_E = East['oMn'].values

In [10]:
def flatten_input(var):
    b = var[0,:,:].flatten()
    for i in range(1,len(var)):
        a = var[i,:,:].flatten()
        b = np.vstack((b,a))
    return b

In [11]:
def create_boundary(rimwidth, boundary_dMn, boundary_oMn, bdy=''):
    
    dMn_O = flatten_input(boundary_dMn)
    oMn_O = flatten_input(boundary_oMn)
    
    print(np.max(boundary_dMn.shape)*rimwidth, dMn_O.shape)
    if bdy=='SE':
        dMn_OBC = np.reshape(dMn_O, (1,50,1,boundary_dMn.shape[1]*rimwidth))
        oMn_OBC = np.reshape(oMn_O, (1,50,1,boundary_dMn.shape[1]*rimwidth))
    else:
        dMn_OBC = np.reshape(dMn_O, (1,50,1,np.max(boundary_dMn.shape)*rimwidth))
        oMn_OBC = np.reshape(oMn_O, (1,50,1,np.max(boundary_dMn.shape)*rimwidth))

    return dMn_OBC, oMn_OBC

In [19]:
def save_BC_file(name, dMn_BC, oMn_BC):
    # Setup the new NetCDF file:
    ncd = nc.Dataset(f'/ocean/brogalla/GEOTRACES/data/{name}', 'w', zlib=True)
    ncd.createDimension('x', np.max(dMn_BC.shape))
    ncd.createDimension('y',1)
    ncd.createDimension('z',50)
    ncd.createDimension('t',None)
    
    print(np.max(dMn_BC.shape))
    
    dMn = ncd.createVariable('dMn', 'float64', ('t','z','y','x'))
    dMn.units = 'mol/L'
    dMn.long_name = 'dissolved Mn'
    dMn[:] = dMn_BC

    oMn = ncd.createVariable('oMn', 'float64', ('t','z','y','x'))
    oMn.units = 'mol/L'
    oMn.long_name = 'oxidised Mn'
    oMn[:] = oMn_BC
    
    ncd.close()
    return

In [13]:
def create_BC(label, bdy_coordinates):
    
    (bdyimin, bdyimax, bdyjmin, bdyjmax) = bdy_coordinates
    
    inif = nc.Dataset('/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-spin-up3_20211012/'+\
                      'ANHA12_EXH006_2002_monthly.nc')
    
    if label=='S':
        ini_dMn = np.array(inif.variables['dissolmn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]
        ini_oMn = np.array(inif.variables['oxidismn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]

        dMn_bdy1 = np.copy(ini_dMn); oMn_bdy1 = np.copy(ini_oMn);
        
    elif label=='SE':
        ini_dMn = np.array(inif.variables['dissolmn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]
        ini_oMn = np.array(inif.variables['oxidismn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]

        dMn_bdy1 = np.copy(ini_dMn); oMn_bdy1 = np.copy(ini_oMn);

    elif label=='W':
        ini_dMn = np.array(inif.variables['dissolmn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]
        ini_oMn = np.array(inif.variables['oxidismn'])[5,0,:,bdyimin-imin:bdyimax-imin,bdyjmin-jmin:bdyjmax-jmin]

        dMn_bdy1 = np.copy(ini_dMn); oMn_bdy1 = np.copy(ini_oMn);

        
    for z in range(0,50):
        # Replace zero values:
        dMn_bdy1[dMn_bdy1 == 0] = np.average(dMn_bdy1[z,:,:])
        oMn_bdy1[oMn_bdy1 == 0] = np.average(oMn_bdy1[z,:,:])

        # Replace ratios that are off:
        dMn_bdy1[(dMn_bdy1/oMn_bdy1) > 50] = np.average(dMn_bdy1[z,:,:])
        oMn_bdy1[(dMn_bdy1/oMn_bdy1) > 50] = np.average(oMn_bdy1[z,:,:])

    print(dMn_bdy1.shape, mbathy.shape)
    dMn_bdy1[(mbathy[:,bdyimin:bdyimax,bdyjmin:bdyjmax] < 0.1)] = 0.2e-9
    oMn_bdy1[(mbathy[:,bdyimin:bdyimax,bdyjmin:bdyjmax] < 0.1)] = 0.2e-9

    # Filter to smoothe out the forcing
    dMn_bdy = ndimage.gaussian_filter(dMn_bdy1, sigma=4, order=0)
    oMn_bdy = ndimage.gaussian_filter(oMn_bdy1, sigma=4, order=0)

    return dMn_bdy, oMn_bdy

In [14]:
rimwidthS  = 10
rimwidthSE = 20
rimwidthW  = 10

In [15]:
N_coordinates  = ((2168, 2178, 201, 798))
E_coordinates  = ((1481, 2178, 778, 798))
SE_coordinates = ((imin+1, imin+36, 680, 680+rimwidthSE))
S_coordinates  = ((imin, imin+rimwidthS, 442, 681))
W_coordinates  = ((1190, 1474, 292, 292+rimwidthW))

# Create boundary conditions:

In [16]:
dMn_west,      oMn_west      = create_BC('W' , W_coordinates)
dMn_south,     oMn_south     = create_BC('S' , S_coordinates)
dMn_southeast, oMn_southeast = create_BC('SE', SE_coordinates)

(50, 284, 10) (50, 2400, 1632)
(50, 10, 239) (50, 2400, 1632)
(50, 35, 20) (50, 2400, 1632)


/home/brogalla/miniconda3/envs/julab/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
/home/brogalla/miniconda3/envs/julab/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide


In [18]:
dMn_west_BC     , oMn_west_BC      = create_boundary(rimwidthW, dMn_west, oMn_west)
dMn_south_BC    , oMn_south_BC     = create_boundary(rimwidthS, dMn_south, oMn_south)
dMn_southeast_BC, oMn_southeast_BC = create_boundary(rimwidthSE, dMn_southeast, oMn_southeast, bdy='SE')

2840 (50, 2840)
2390 (50, 2390)
1000 (50, 700)


In [20]:
save_BC_file('Mn_Southeast_OBC_extended.nc', dMn_southeast_BC, oMn_southeast_BC)

700


In [21]:
save_BC_file('Mn_South_OBC_extended.nc', dMn_south_BC, oMn_south_BC)

2390


In [22]:
save_BC_file('Mn_West_OBC_extended.nc', dMn_west_BC, oMn_west_BC)

2840
